# Extensions

The purpose of this notebook is to test everything related to generating extensions among hits

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('..', 'hypedsearch', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import testing_utils
import database

import operator

ppm_tolerance = 20
max_peptide_length = 20

import matplotlib.pyplot as plt

In [2]:
datasets = testing_utils.define_data()

dataset = datasets[0]

input_spectra_path = dataset[0]
input_spectra, boundaries = testing_utils.preprocess_input_spectra(input_spectra_path, ppm_tolerance)

correct_sequences = testing_utils.generate_truth_set(datasets[0])

path = dataset[2]
db = database.build(path)

Loading spectra...
Done


In [3]:
matched_masses_b, matched_masses_y, database = testing_utils.modified_match_masses(boundaries, db, max_peptide_length)
# matched_masses_b, matched_masses_y, database = match_masses(boundaries, db, max_peptide_length)
print('Finished matching masses')

On protein 279/279 [100%]
Sorting the set of protein masses...
Sorting the set of protein masses done
Performing Merge
Done
Finished matching masses


# Getting initial hits

In [24]:
spectrum_num = 100

correct_sequence = correct_sequences[spectrum_num]
print(correct_sequence)

input_spectrum = input_spectra[spectrum_num]

DGPVQGTIHFEQKASGEPVVLSGQITGLTEGQHGFHVHQYG


In [25]:
b_hits, y_hits, b_set, y_set, misses = testing_utils.find_hits(boundaries, input_spectrum, spectrum_num, matched_masses_b, matched_masses_y)
correct_hits = testing_utils.append_correct_hits(correct_sequence, input_spectrum, ppm_tolerance)
testing_utils.write_hits(b_hits, y_hits)
ion = 'b'
testing_utils.create_clusters(ion)
b_sorted_clusters = testing_utils.sort_clusters_by_post_prob(ion, boundaries, matched_masses_b, matched_masses_y)
ion = 'y'
testing_utils.create_clusters(ion)
y_sorted_clusters = testing_utils.sort_clusters_by_post_prob(ion, boundaries, matched_masses_b, matched_masses_y)

Done


# Printing hits

In [26]:
# b_sorted_clusters = sorted(b_sorted_clusters, key=operator.attrgetter('post_prob', 'score', 'pid', 'prior'), reverse = True)
b_sorted_clusters = sorted(b_sorted_clusters, key=operator.attrgetter('score', 'post_prob', 'pid', 'prior'), reverse = True)
testing_utils.write_b_sorted_cluster(b_sorted_clusters)
for i in range(0, 50):
    x = b_sorted_clusters[i]
    post_prob = x.post_prob
    score = x.score
    seq = x.seq
    indices = x.indices
    print(post_prob, score, seq)

0.4825931870875691 3 GSQANS
0.46677821156348764 3 GSQFFICT
0.4510763209393346 3 PVGSPSR
0.4305324715297014 3 SGQADDERVREYHLL
0.42809046908769893 3 GSGAVALCPE
0.42769607843137253 3 GSQYRVESMMLRIAKPM
0.42769607843137253 3 ANSWNLDWGDNGFFKIL
0.42649162917619293 3 GQGQSPSIRQ
0.42497691597414583 3 QGSPVVQMAEDAVDGER
0.4223938223938224 3 QGSPGAWAPLDPTSGSSA
0.42117433930093773 3 GQSTVPPCTASPEPVKAAEQ
0.416856336932205 3 GSQMEDARRPAEPSLGGLPQ
0.4107948442534909 3 GSAGLMLVEFFAPWCG
0.3996786444639205 3 PVPVDCSDEAGNSALQLA
0.3744369369369369 3 SNSQQAYQEAFEI
0.3462974111980735 3 GQVLPEMEIHLQTDAKKGT
0.3142857142857143 3 GSPGSAGLVQHLEEYAAT
0.31201298701298696 3 SSNGFLIDGYPREVKQGE
0.30300736790803023 3 VPEMKPSMFDVSRELGSSV
0.2988665811585478 3 GQVRLTYSTGESNTVVSPTV
0.17540849673202613 3 IFVGTPYYGYVYEDTLP
0.15911553616661975 3 QVNQSLASMPPMNPATTLP
0.14827579466745225 3 MAEVDAAMAARPHSIDGRVV
0.3698769395398609 2 GQS
0.3698769395398609 2 QGS
0.3698769395398609 2 GQS
0.3698769395398609 2 NAS
0.3698769395398609 2 

In [27]:
testing_utils.write_y_sorted_cluster(y_sorted_clusters)  
# y_sorted_clusters = sorted(y_sorted_clusters, key=operator.attrgetter('post_prob', 'score', 'pid', 'prior'), reverse = True)
y_sorted_clusters = sorted(y_sorted_clusters, key=operator.attrgetter('score', 'post_prob', 'pid', 'prior'), reverse = True)
for i in range(0, 50):
    x = y_sorted_clusters[i]
    post_prob = x.post_prob
    score = x.score
    seq = x.seq
    indices = x.indices
    print(post_prob, score, seq)

0.41561085972850675 3 YEAGEKRWGTDEVK
0.39088600688323677 3 SRPCTPPQ
0.387228531579996 3 DVHPSDLKPKGDDKDPSK
0.3844489981785064 3 GLGNKTYEHFNAMGK
0.35945075380221825 3 GRPSGEAFVELESEDEVK
0.3586080586080586 3 VVWTSEYDPLASNPGWKK
0.35402930402930405 3 VCPTEIIAFSDHAEDFRK
0.35402930402930405 3 VTVTVLDVNDNRPEFTMK
0.18003292889544734 3 TGLTEGQHGFHVHQYG
0.44019607843137254 2 FVK
0.44019607843137254 2 VFK
0.44019607843137254 2 FVK
0.44019607843137254 2 VFK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 VFK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 VFK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 VFK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 FVK
0.44019607843137254 2 VFK
0.44019607843137254 2 VFK
0.44019607843137254 2 FVK
0.44

# To Filter data by parent prot

I want to be able to only view the b or y hits from a certain protein

In [28]:
# b_hits
target_pid = 274
b_target_clusters = []
for cluster in b_sorted_clusters:
    if cluster.pid == target_pid:
        b_target_clusters.append(cluster)

for cluster in b_target_clusters:
    assessment, _ = testing_utils.is_good_hit(cluster.seq, 'b', correct_sequence)
    non_indices = str(cluster.score) + '\t' + str(cluster.post_prob) + '\t' + str(cluster.pid) + '\t' + cluster.seq + '\t' + str(assessment)
    print(non_indices + '\t'+ '\t'.join([str(o) for o in cluster.indices]))


1	0.20083892617449667	274	GS	False	(70, 71, 'GS', '145.0601806640625')
1	0.14363173619564013	274	LCGP	False	(30, 33, 'LCGP', '186.08746337890625')
1	0.11385083713850837	274	EPKPTQ	False	(18, 23, 'EPKPTQ', '341.1816101074219')
1	0.07366570789389582	274	LGGSPGDLQTL	False	(68, 78, 'LGGSPGDLQTL', '1039.521484375')
1	0.048765803732691154	274	RREVEDPQVEQLELGGSPG	False	(55, 73, 'RREVEDPQVEQLELGGSPG', '1039.0203857421875')
1	0.04772727272727273	274	DPQVEQLELGGSPGDLQTL	False	(60, 78, 'DPQVEQLELGGSPGDLQTL', '989.4848022460938')


In [29]:
# y_hits
y_target_clusters = []
for cluster in y_sorted_clusters:
    if cluster.pid == target_pid:
        y_target_clusters.append(cluster)

for cluster in y_target_clusters:
    assessment, _ = testing_utils.is_good_hit(cluster.seq, 'y', correct_sequence)
    non_indices = str(cluster.score) + '\t' + str(cluster.post_prob) + '\t' + str(cluster.pid) + '\t' + cluster.seq + '\t' + str(assessment)
    print(non_indices + '\t'+ '\t'.join([str(o) for o in cluster.indices]))

2	0.44019607843137254	274	FVK	False	(25, 27, 'FVK', '197.12884521484375')	(27, 27, 'K', '74.06013488769531')
1	0.25015087507543754	274	K	False	(20, 20, 'K', '74.06013488769531')
1	0.25015087507543754	274	K	False	(53, 53, 'K', '74.06013488769531')
1	0.25015087507543754	274	K	False	(86, 86, 'K', '74.06013488769531')
1	0.11453576864535768	274	ALYLVC	False	(38, 43, 'ALYLVC', '341.1816101074219')
1	0.06428801028608165	274	EALYLVCGERGFFY	False	(37, 50, 'EALYLVCGERGFFY', '833.8908081054688')
1	0.05867082035306334	274	EALYLVCGERGFFYT	False	(37, 51, 'EALYLVCGERGFFYT', '884.4120483398438')
1	0.054694835680751175	274	VEDPQVEQLELGGSPGD	False	(58, 74, 'VEDPQVEQLELGGSPGD', '884.9159545898438')
1	0.05128205128205128	274	LVEALYLVCGERGFFYTP	False	(35, 52, 'LVEALYLVCGERGFFYTP', '1039.0203857421875')


# Finding non-hybrid interesting Combos

In [30]:
def get_top(b_clusters, y_clusters, top_num):
    filtered_b = []
    filtered_y = []
    b_len = top_num if len(b_clusters) >= top_num else len(b_clusters)
    y_len = top_num if len(y_clusters) >= top_num else len(y_clusters)
    for x in range(0,b_len):
        filtered_b.append(b_clusters[x])
    for x in range(0,y_len):
        filtered_y.append(y_clusters[x])
    return filtered_b, filtered_y

In [31]:
# b side starting. Ideally, we would probably pick the higher scoring side to start
filtered_b, filtered_y = get_top(b_target_clusters, y_target_clusters, 50)
target_precursor = input_spectrum.precursor_mass
#Start with printing overlapping. Then will incorportate boundary overlaps between last of b and first of y
for b_cluster in filtered_b:
    interesting_combos = []
    for y_cluster in filtered_y:
        if b_cluster.start <= y_cluster.end:
            interesting_combos.append(b_cluster.seq + '-' + y_cluster.seq)
    print(interesting_combos)
# # Calculating start and end indices for each interval
# for b_cluster in filtered_b:
#     interesting_combos = []
#     b_index_set = set()
#     for index in b_cluster.indices:
#         index = index.replace('(', '')
#         index = index.replace(')', '')
#         A = index.rstrip().split(',')
#         b_index_set.add(int(A[0]))
#         b_index_set.add(int(A[1]))
    
#     b_start_pos = min(b_index_set)
#     b_end_pos = max(b_index_set)
    
#     for y_cluster in filtered_y:
#         y_index_set = set()
#         for index in y_cluster.indices:
#             index = index.replace('(', '')
#             index = index.replace(')', '')
#             A = index.rstrip().split(',')
#             y_index_set.add(int(A[0]))
#             y_index_set.add(int(A[1]))
            
#         y_start_pos = min(y_index_set)
#         y_end_pos = max(y_index_set)
        
#         if (b_start_pos <= y_start_pos) and (b_end_pos == y_start_pos - 1):
#             interesting_combos.append(b_cluster)
#             interesting_combos.append(y_cluster)
    
#     print(interesting_combos)

['GS-K', 'GS-VEDPQVEQLELGGSPGD']
['LCGP-K', 'LCGP-K', 'LCGP-ALYLVC', 'LCGP-EALYLVCGERGFFY', 'LCGP-EALYLVCGERGFFYT', 'LCGP-VEDPQVEQLELGGSPGD', 'LCGP-LVEALYLVCGERGFFYTP']
['EPKPTQ-FVK', 'EPKPTQ-K', 'EPKPTQ-K', 'EPKPTQ-K', 'EPKPTQ-ALYLVC', 'EPKPTQ-EALYLVCGERGFFY', 'EPKPTQ-EALYLVCGERGFFYT', 'EPKPTQ-VEDPQVEQLELGGSPGD', 'EPKPTQ-LVEALYLVCGERGFFYTP']
['LGGSPGDLQTL-K', 'LGGSPGDLQTL-VEDPQVEQLELGGSPGD']
['RREVEDPQVEQLELGGSPG-K', 'RREVEDPQVEQLELGGSPG-VEDPQVEQLELGGSPGD']
['DPQVEQLELGGSPGDLQTL-K', 'DPQVEQLELGGSPGDLQTL-VEDPQVEQLELGGSPGD']


# Finding optimal "hybrid" combos

* Hybrid is in quotation marks because all outputs will be a hybrid and then we can check if it is a non-hybrid

In [32]:
def parse_indices(index_set):
    indices = []
    for index in index_set:
        string = str(index)
        A = string.rstrip().split(',')
        start = A[0]
        end = A[1]
        seq = A[2]
        mz = A[3]
        disallowed_characters = " ()\'"
        for character in disallowed_characters:
            start = start.replace(character, "")
            end = end.replace(character, "")
            seq = seq.replace(character, "")
            mz = mz.replace(character, "")
        
        target_tuple = (int(start), int(end), seq, float(mz))
        indices.append(target_tuple)
    
    
    return indices

def calc_combined_score(b_indices, y_indices, b_score, y_score):
    masses = []
    combined_score = 0
    for index in b_indices:
        current_mass = float(index[3])
        if current_mass not in masses:
            masses.append(current_mass)
            combined_score = combined_score + 1
        else:
            combined_score = combined_score - 2
    for index in y_indices:
        current_mass = float(index[3])
        if current_mass not in masses:
            masses.append(current_mass)
            combined_score = combined_score + 1
        else:
            combined_score = combined_score - 2
    return combined_score

In [33]:
# b side starting. Ideally, we would probably pick the higher scoring side to start
filtered_b, filtered_y = get_top(b_sorted_clusters, y_sorted_clusters, 50)
target_precursor = input_spectrum.precursor_mass
#Start with printing overlapping. Then will incorportate boundary overlaps between last of b and first of y
interesting_combos = []
for b_cluster in filtered_b:
    for y_cluster in filtered_y:
        if b_cluster.start <= y_cluster.end:
            seq = b_cluster.seq + '-' + y_cluster.seq
            b_indices = parse_indices(b_cluster.indices)
            y_indices = parse_indices(y_cluster.indices)
            score = calc_combined_score(b_indices, y_indices, b_cluster.score, y_cluster.score)
            tup = (seq, score)
            interesting_combos.append(tup)
interesting_combos.sort(key=lambda a: a[1], reverse=True)
print(interesting_combos)
            
            
# # Calculating start and end indices for each interval
# for b_cluster in filtered_b:
#     interesting_combos = []
#     b_index_set = set()
#     for index in b_cluster.indices:
#         index = index.replace('(', '')
#         index = index.replace(')', '')
#         A = index.rstrip().split(',')
#         b_index_set.add(int(A[0]))
#         b_index_set.add(int(A[1]))
    
#     b_start_pos = min(b_index_set)
#     b_end_pos = max(b_index_set)
    
#     for y_cluster in filtered_y:
#         y_index_set = set()
#         for index in y_cluster.indices:
#             index = index.replace('(', '')
#             index = index.replace(')', '')
#             A = index.rstrip().split(',')
#             y_index_set.add(int(A[0]))
#             y_index_set.add(int(A[1]))
            
#         y_start_pos = min(y_index_set)
#         y_end_pos = max(y_index_set)
        
#         if (b_start_pos <= y_start_pos) and (b_end_pos == y_start_pos - 1):
#             interesting_combos.append(b_cluster)
#             interesting_combos.append(y_cluster)
    
#     print(interesting_combos)

[('GSQFFICT-GLGNKTYEHFNAMGK', 6), ('GSQFFICT-VVWTSEYDPLASNPGWKK', 6), ('GSQFFICT-VTVTVLDVNDNRPEFTMK', 6), ('SGQADDERVREYHLL-GLGNKTYEHFNAMGK', 6), ('SGQADDERVREYHLL-VVWTSEYDPLASNPGWKK', 6), ('SGQADDERVREYHLL-VTVTVLDVNDNRPEFTMK', 6), ('GSGAVALCPE-GLGNKTYEHFNAMGK', 6), ('GSGAVALCPE-GRPSGEAFVELESEDEVK', 6), ('GSGAVALCPE-VVWTSEYDPLASNPGWKK', 6), ('GSGAVALCPE-VCPTEIIAFSDHAEDFRK', 6), ('GSGAVALCPE-VTVTVLDVNDNRPEFTMK', 6), ('ANSWNLDWGDNGFFKIL-VVWTSEYDPLASNPGWKK', 6), ('QGSPGAWAPLDPTSGSSA-GLGNKTYEHFNAMGK', 6), ('QGSPGAWAPLDPTSGSSA-VVWTSEYDPLASNPGWKK', 6), ('QGSPGAWAPLDPTSGSSA-VCPTEIIAFSDHAEDFRK', 6), ('QGSPGAWAPLDPTSGSSA-VTVTVLDVNDNRPEFTMK', 6), ('QGSPGAWAPLDPTSGSSA-TGLTEGQHGFHVHQYG', 6), ('GSAGLMLVEFFAPWCG-GRPSGEAFVELESEDEVK', 6), ('GSAGLMLVEFFAPWCG-VVWTSEYDPLASNPGWKK', 6), ('GSAGLMLVEFFAPWCG-VCPTEIIAFSDHAEDFRK', 6), ('GSAGLMLVEFFAPWCG-VTVTVLDVNDNRPEFTMK', 6), ('SNSQQAYQEAFEI-VVWTSEYDPLASNPGWKK', 6), ('SNSQQAYQEAFEI-VTVTVLDVNDNRPEFTMK', 6), ('GQVLPEMEIHLQTDAKKGT-GLGNKTYEHFNAMGK', 6), ('GQVLPEM